To do:
- create per division mean value of:
    - rounds
    - age
    - bouts
    - age
    - losses
    - draws
    - height
    - reach
        - reach relative to height
    - time since debut >> == (current date - debut)

- for values missing in:
    - age:
        - no data = average age of division 
    - reach:
        - reach == (height + reach relative to height for division)
    - height:
        - average weight of division and replace missing values with this...
    - stance:
        unclear...

age - missing = 19/793 - pct missing = 2.4%
stance - missing = 82/793 - pct missing = 10.34%    
height - missing = 98/793 - pct missing = 12.36%    
reach - missing = 380/793 - pct missing = 47.92%    
birth place - missing = 79/793 - pct missing = 9.96%

In [10]:
import pandas as pd
import numpy as np

boxrec_data = pd.read_csv('boxrec_tables.csv')
df = pd.DataFrame(boxrec_data)


In [11]:
df.replace(to_replace='None', value=np.nan, regex=True, inplace=True)

In [12]:
df.insert(3, 'div index', '')


In [13]:
div_list = ['heavy', 'cruiser', 'lightheavy', 'supermiddle', 'middle', 'superwelter', 'welter', 'superlight', 'light', 'feather', 'superbantam', 'bantam', 'superfly', 'fly', 'lightfly', 'minimum']

x = 0
for value in df['division']:
    df.at[x, 'div index'] = div_list.index(value)
    x += 1

In [14]:
ds = df.sort_values(by=['div index', 'division rating'], ascending=True)
rounds = df.sort_values(by=['rounds'], ascending = False)

In [74]:
# ds.dtypes 
# Can see age/height/reach need to be turned to float64s
cols_to_convert = ['age', 'height', 'reach']

for col in cols_to_convert:
    ds[col] = ds[col].astype(float)


In [77]:
average_by_division = ds.groupby(['division']).mean(numeric_only=True)

average_everything_numeric = ds.mean(numeric_only=True)

,division rating,bouts,rounds,KOs,age,height,reach,wins,losses,draws,KO wins,KO losses
division,,,,,,,,,,,,
bantam,25.500000,22.740000,120.220000,53.684800,28.250000,166.292683,169.560000,19.760000,2.300000,0.520000,12.460000,0.540000
cruiser,25.660000,23.360000,121.140000,61.703000,32.360000,187.047619,191.846154,20.840000,2.180000,0.260000,14.100000,0.840000
feather,25.540000,23.500000,134.260000,49.171400,27.714286,170.125000,171.966667,21.260000,1.720000,0.460000,11.500000,0.580000
fly,25.520000,19.280000,103.600000,48.661800,27.520000,163.538462,166.631579,16.700000,2.060000,0.460000,9.800000,0.520000
heavy,26.666667,23.250000,109.916667,69.136250,33.187500,193.042553,198.516129,21.062500,1.750000,0.291667,15.645833,0.854167
light,24.600000,25.860000,151.660000,55.727400,29.640000,172.304348,174.066667,23.400000,1.900000,0.440000,13.780000,0.680000
lightfly,25.500000,18.340000,99.280000,47.501400,26.897959,161.567568,162.866667,14.840000,2.720000,0.660000,8.340000,0.960000
lightheavy,25.191489,21.085106,110.191489,56.482340,31.425532,184.860465,187.315789,19.063830,1.744681,0.255319,11.829787,0.872340
middle,25.612245,23.734694,126.530612,57.970612,30.857143,181.046512,184.280000,21.285714,1.816327,0.591837,13.673469,0.714286
